In [1]:
try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om
    
from openmdao.utils.assert_utils import assert_near_equal

# Problem Recording

You might also want to attach a recorder to the `Problem` itself. This allows you to record an arbitrary case at a point of your choosing. This feature can be useful if you only record a limited number of variables during the run but would like to see a more complete list of values after the run.

The options are a subset of those for driver recording.

In [2]:
om.show_options_table("openmdao.core.problem.Problem", recording_options=True)

Option,Default,Acceptable Values,Acceptable Types,Description
excludes,[],N/A,['list'],Patterns for vars to exclude in recording (processed post-includes). Uses fnmatch wildcards
includes,['*'],N/A,['list'],Patterns for variables to include in recording. Uses fnmatch wildcards
record_abs_error,True,"[True, False]",['bool'],Set to True to record absolute error of model nonlinear solver
record_constraints,True,"[True, False]",['bool'],Set to True to record constraints at the problem level
record_derivatives,False,"[True, False]",['bool'],Set to True to record derivatives for the problem level
record_desvars,True,"[True, False]",['bool'],Set to True to record design variables at the problem level
record_inputs,False,"[True, False]",['bool'],Set True to record inputs at the problem level.
record_objectives,True,"[True, False]",['bool'],Set to True to record objectives at the problem level
record_outputs,True,"[True, False]",['bool'],Set True to record outputs at the problem level.
record_rel_error,True,"[True, False]",['bool'],Set to True to record relative error of model nonlinear solver


> **_Note:_**  Note that the `excludes` option takes precedence over the `includes` option.

## Problem Recording Example

In [3]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

import numpy as np

prob = om.Problem(model=SellarDerivatives())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9)

prob.add_recorder(om.SqliteRecorder("cases.sql"))

prob.recording_options['includes'] = []
prob.recording_options['record_objectives'] = True
prob.recording_options['record_constraints'] = True
prob.recording_options['record_desvars'] = True

prob.setup()
prob.run_driver()

NL: NLBGS Converged in 8 iterations
NL: NLBGS Converged in 1 iterations
NL: NLBGS Converged in 9 iterations
NL: NLBGS Converged in 10 iterations
NL: NLBGS Converged in 10 iterations
NL: NLBGS Converged in 9 iterations
NL: NLBGS Converged in 6 iterations
Optimization terminated successfully    (Exit mode 0)
            Current function value: [3.18339395]
            Iterations: 6
            Function evaluations: 6
            Gradient evaluations: 6
Optimization Complete
-----------------------------------


False

In [4]:
prob.record('final')
prob.cleanup()

cr = om.CaseReader("cases.sql")

# get list of cases recorded on problem
problem_cases = cr.list_cases('problem')

problem
final


In [5]:
assert problem_cases == ['final']

In [6]:
# get list of output variables recorded on problem
cr.list_source_vars('problem')

inputs,outputs,residuals
,con1,
,obj,
,con2,
,z,
,x,


{'inputs': [], 'outputs': ['con1', 'obj', 'con2', 'z', 'x'], 'residuals': []}

In [7]:
problem_vars = cr.list_source_vars('problem', out_stream=None)
assert sorted(problem_vars['outputs']) == ['con1', 'con2', 'obj', 'x', 'z']

In [8]:
# get the recorded case
case = cr.get_case('final')

In [9]:
case.get_objectives()

{'obj': array([3.18339395])}

In [10]:
assert_near_equal(case.get_objectives()['obj'], 3.18339395, 1e-8)

5.41351690502717e-10

In [11]:
case.get_design_vars()

{'z': array([1.97763888e+00, 5.87897722e-15]), 'x': array([0.])}

In [12]:
des_vars = case.get_design_vars()
assert_near_equal(des_vars['z'], [1.97763888, 5.87897722e-15], 1e-8)
assert_near_equal(des_vars['x'], 0., 1e-8)

0.0

In [13]:
case.get_constraints()

{'con1': array([-8.38054071e-11]), 'con2': array([-20.24472223])}

In [14]:
constraints = case.get_constraints()
assert_near_equal(constraints['con1'], -8.38054071e-11, 1e-8)
assert_near_equal(constraints['con2'], -20.24472223, 1e-8)

1.4953032152054424e-10